In [ ]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [ ]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0].transpose()
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:].transpose()
    data_x = normalize(data_x)
    return data_x, data_y

In [ ]:
training_data_x, training_data_y = features_and_labels("../archive/exoTrain.csv")

#set hyperparameters & variables
learning_rate = 0.003
epochs = 500
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

In [ ]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })

    if t % display_step == 0:
        print("t = %g, loss = %g, W = %s, b = %s" % (t, current_loss, str(current_W), str(current_b)))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "W=", session.run(W), "b=", session.run(b), '\n')

t = 0, loss = 0.00363672, W = [[1.20844725e-05 1.08624172e-05 6.64995059e-06 ... 2.96250755e-06
  5.31442720e-06 6.92352614e-06]], b = [2.1820326e-05]
t = 5, loss = 0.00356816, W = [[7.8847261e-05 7.0592912e-05 4.6259072e-05 ... 2.3039685e-05
  3.6960933e-05 4.6242436e-05]], b = [0.00013327]
t = 10, loss = 0.00352213, W = [[1.4821738e-04 1.3214620e-04 8.8468623e-05 ... 4.5107339e-05
  7.0611262e-05 8.7744906e-05]], b = [0.00024347]
t = 15, loss = 0.00348207, W = [[2.1559105e-04 1.9126640e-04 1.2876095e-04 ... 6.7038280e-05
  1.0422215e-04 1.2963693e-04]], b = [0.00035196]
t = 20, loss = 0.00344504, W = [[2.8007483e-04 2.4720989e-04 1.6630910e-04 ... 8.8442190e-05
  1.3736074e-04 1.7138754e-04]], b = [0.00045895]
t = 25, loss = 0.00341012, W = [[0.00034164 0.00030004 0.00020114 ... 0.00010924 0.00016991 0.00021277]], b = [0.00056453]
t = 30, loss = 0.00337687, W = [[0.00040044 0.00034996 0.00023348 ... 0.00012944 0.00020185 0.00025368]], b = [0.00066875]
t = 35, loss = 0.00334507, W = [

In [ ]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [ ]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [ ]:
test_x, test_y = features_and_labels("../archive/exoTest.csv")

predicted_y = session.run(W) * test_x + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  52.63157894736842 %


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c58af11-5d4f-40f3-a808-e868f2c28485' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>